In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
done

# All requested packages already installed.

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
done

# All requested packages already installed.

Libraries imported.


In [2]:
#swiss_data = pd.read_json("https://geo.nyu.edu/download/file/stanford-cm591kd0445-geojson.json")
#swiss_data = pd.read_json("https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/CH/zipcodes.ch.json")

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
zurich_data = pd.read_json(body, orient='values')
zurich_data.head()

,type,name,features
0,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [5]:
zurich_data_features = zurich_data['features']
zurich_data_features.head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

In [6]:
#swiss_data.sort_values('place',ascending=False)
#swiss_data

In [7]:
#zurich_data = swiss_data[swiss_data['place'] == 'Zürich'].reset_index(drop=True)
#zurich_data

In [8]:
# define the dataframe columns
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [10]:
for data in zurich_data_features:
    neighborhood_name = data['properties']['name'] 

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': "Zürich",
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                           ignore_index=True)
    

In [11]:
neighborhoods.shape

(22, 4)

In [12]:
print('The dataframe has {} neighborhoods.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [13]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Zürich,Hirslanden,47.363057,8.570403
1,Zürich,Witikon,47.359977,8.600887
2,Zürich,Leimbach,47.331480,8.509313
3,Zürich,Wollishofen,47.341491,8.532956
4,Zürich,Enge,47.360931,8.533140
5,Zürich,Riesbach,47.354570,8.556241
6,Zürich,Affoltern,47.423281,8.509221
7,Zürich,Seebach,47.423695,8.542830
8,Zürich,Oerlikon,47.409348,8.545117
9,Zürich,Höngg,47.407904,8.496424


In [14]:
address = 'Zurich, Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zurich are 47.3744489, 8.5410422.


In [15]:
# create map of Zurich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

## Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q' # your Foursquare ID
CLIENT_SECRET = 'F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q
CLIENT_SECRET:F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ


## Exploring Zurich neighborhoods

In [17]:
neighborhoods.loc[0, 'Neighborhood']

'Hirslanden'

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hirslanden are 47.3630568387, 8.5704034113.


# Get the top 100 venues that are in Hirslanden within a radius of 500 meters.

In [19]:
# Create the GET request URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q&client_secret=F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ&v=20180605&ll=47.3630568387,8.5704034113&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6064d55804959b5a1f8ee7de'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kreis 7',
  'headerFullLocation': 'Kreis 7, Zürich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 47.3675568432, 'lng': 8.577034546600979},
   'sw': {'lat': 47.358556834199995, 'lng': 8.56377227599902}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6b93162d9976b016990c3a',
       'name': 'Marcellino, Pane e Vino',
       'location': {'address': 'Forchstrasse 168',
        'lat': 47.360202,
        'lng': 8.565569,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.360202,
          'lng': 8.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,"Marcellino, Pane e Vino",Mediterranean Restaurant,47.360202,8.565569
1,Wilder Mann,Swiss Restaurant,47.360341,8.566796
2,Ristorante Antiquario,Italian Restaurant,47.360676,8.566550
3,Klusplatz,Plaza,47.364025,8.566751
4,Bistro Le Puy,French Restaurant,47.359382,8.568897


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Explore neighborhoods in Zurich

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
zurich_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude']
                                )

Hirslanden
Witikon
Leimbach
Wollishofen
Enge
Riesbach
Affoltern
Seebach
Oerlikon
Höngg
Unterstrass
Wipkingen
Schwamendingen
Oberstrass
Altstetten
Aussersihl
Altstadt
Fluntern
Hottingen
Albisrieden
Wiedikon
Industriequartier


In [27]:
print(zurich_venues.shape)
zurich_venues.head()

(506, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant


In [28]:
# Venues per neighborhood
zurich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Affoltern,4,4,4,4,4,4
Albisrieden,4,4,4,4,4,4
Altstadt,88,88,88,88,88,88
Altstetten,17,17,17,17,17,17
Aussersihl,59,59,59,59,59,59
Enge,31,31,31,31,31,31
Fluntern,6,6,6,6,6,6
Hirslanden,9,9,9,9,9,9
Hottingen,2,2,2,2,2,2


In [30]:
# Check unique categories we received for Zurich venues
print('There are {} unique categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 144 unique categories.


## 3. Analyze Each Neighborhood